# Capstone Project- The Battle of the Neighborhoods

### Applied Data Science Capstone by IBM/Coursera

### Introduction
In this project I will be analyzing income inequality disparities within Boston and how this manifests in its neighborhoods. I will be using a combination of Four square location data as well as Boston income data

### Per Capita Income by Boston Neighborhood

In [19]:
import numpy as np
import folium
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json

from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

from bs4 import BeautifulSoup

import xml

import folium

print('Libraries Imported.')

Libraries Imported.


In [20]:
url_mass = "https://en.wikipedia.org/wiki/Boston"
wiki_mass_url = requests.get(url_mass)
wiki_mass_data = pd.read_html(wiki_mass_url.text)
wiki_mass_data = wiki_mass_data[9]
wiki_mass_data

,Rank,ZIP code (ZCTA),Per capitaincome,Medianhouseholdincome,Medianfamilyincome,Population,Number ofhouseholds
0,1.0,02110 (Financial District),"$152,007","$123,795","$196,518",1486,981
1,2.0,02199 (Prudential Center),"$151,060","$107,159","$146,786",1290,823
2,3.0,02210 (Fort Point),"$93,078","$111,061","$223,411",1905,1088
3,4.0,02109 (North End),"$88,921","$128,022","$162,045",4277,2190
4,5.0,02116 (Back Bay/Bay Village),"$81,458","$87,630","$134,875",21318,10938
5,6.0,02108 (Beacon Hill/Financial District),"$78,569","$95,753","$153,618",4155,2337
6,7.0,02114 (Beacon Hill/West End),"$65,865","$79,734","$169,107",11933,6752
7,8.0,02111 (Chinatown/Financial District/Leather Di...,"$56,716","$44,758","$88,333",7616,3390
8,9.0,02129 (Charlestown),"$56,267","$89,105","$98,445",17052,8083
9,10.0,02467 (Chestnut Hill),"$53,382","$113,952","$148,396",22796,6351


### Cleaning Up the Table to Make the Data Clearer

Starting by dropping the Percapitaincome and Medianfamilyincome columns. Then sorted by median household income.

In [21]:
wiki_mass_data.drop(['Per capitaincome', 'Medianfamilyincome'], axis=1,inplace=True)
wiki_mass_data.head()

,Rank,ZIP code (ZCTA),Medianhouseholdincome,Population,Number ofhouseholds
0,1.0,02110 (Financial District),"$123,795",1486,981
1,2.0,02199 (Prudential Center),"$107,159",1290,823
2,3.0,02210 (Fort Point),"$111,061",1905,1088
3,4.0,02109 (North End),"$128,022",4277,2190
4,5.0,02116 (Back Bay/Bay Village),"$87,630",21318,10938


In [22]:
wiki_mass_data.sort_values(by='Medianhouseholdincome', ascending=True)

,Rank,ZIP code (ZCTA),Medianhouseholdincome,Population,Number ofhouseholds
1,2.0,02199 (Prudential Center),"$107,159",1290,823
2,3.0,02210 (Fort Point),"$111,061",1905,1088
9,10.0,02467 (Chestnut Hill),"$113,952",22796,6351
0,1.0,02110 (Financial District),"$123,795",1486,981
3,4.0,02109 (North End),"$128,022",4277,2190
28,25.0,"02115 (Back Bay, Longwood, Museum of Fine Arts...","$23,677",29178,9958
31,28.0,02119 (Roxbury),"$27,051",24237,9769
32,29.0,02121 (Dorchester-Mount Bowdoin),"$30,419",26801,9739
30,27.0,02215 (Fenway-Kenmore),"$30,823",23719,7995
33,30.0,02120 (Mission Hill),"$32,367",13217,4509


In [23]:
wiki_mass_data = wiki_mass_data.rename(index=str, columns={'Medianhouseholdincome': 'Median Household Income', 'ZIP code (ZCTA)': 'Zip_Code','Number ofhouseholds':'Number of Households'})

In [24]:
wiki_mass_data.head()

,Rank,Zip_Code,Median Household Income,Population,Number of Households
0,1.0,02110 (Financial District),"$123,795",1486,981
1,2.0,02199 (Prudential Center),"$107,159",1290,823
2,3.0,02210 (Fort Point),"$111,061",1905,1088
3,4.0,02109 (North End),"$128,022",4277,2190
4,5.0,02116 (Back Bay/Bay Village),"$87,630",21318,10938


In [25]:
wiki_mass_data[['Zip Code', 'District']] = wiki_mass_data.Zip_Code.str.split("(", expand=True)

In [26]:
wiki_mass_data.head()

,Rank,Zip_Code,Median Household Income,Population,Number of Households,Zip Code,District
0,1.0,02110 (Financial District),"$123,795",1486,981,02110,Financial District)
1,2.0,02199 (Prudential Center),"$107,159",1290,823,02199,Prudential Center)
2,3.0,02210 (Fort Point),"$111,061",1905,1088,02210,Fort Point)
3,4.0,02109 (North End),"$128,022",4277,2190,02109,North End)
4,5.0,02116 (Back Bay/Bay Village),"$87,630",21318,10938,02116,Back Bay/Bay Village)


In [27]:
wiki_mass_data.drop(['Zip_Code'], axis=1, inplace=True)

In [28]:
wiki_mass_data.head()

,Rank,Median Household Income,Population,Number of Households,Zip Code,District
0,1.0,"$123,795",1486,981,02110,Financial District)
1,2.0,"$107,159",1290,823,02199,Prudential Center)
2,3.0,"$111,061",1905,1088,02210,Fort Point)
3,4.0,"$128,022",4277,2190,02109,North End)
4,5.0,"$87,630",21318,10938,02116,Back Bay/Bay Village)


In [29]:
wiki_mass_data = wiki_mass_data[['Zip Code', 'District', 'Median Household Income', 'Population']]

In [30]:
wiki_mass_data.head()

,Zip Code,District,Median Household Income,Population
0,02110,Financial District),"$123,795",1486
1,02199,Prudential Center),"$107,159",1290
2,02210,Fort Point),"$111,061",1905
3,02109,North End),"$128,022",4277
4,02116,Back Bay/Bay Village),"$87,630",21318


In [31]:
wiki_mass_data['District'] = wiki_mass_data['District'].str.replace(')'," ")
wiki_mass_data.head()

,Zip Code,District,Median Household Income,Population
0,02110,Financial District,"$123,795",1486
1,02199,Prudential Center,"$107,159",1290
2,02210,Fort Point,"$111,061",1905
3,02109,North End,"$128,022",4277
4,02116,Back Bay/Bay Village,"$87,630",21318


In [32]:
wiki_mass_data

,Zip Code,District,Median Household Income,Population
0,02110,Financial District,"$123,795",1486
1,02199,Prudential Center,"$107,159",1290
2,02210,Fort Point,"$111,061",1905
3,02109,North End,"$128,022",4277
4,02116,Back Bay/Bay Village,"$87,630",21318
5,02108,Beacon Hill/Financial District,"$95,753",4155
6,02114,Beacon Hill/West End,"$79,734",11933
7,02111,Chinatown/Financial District/Leather District,"$44,758",7616
8,02129,Charlestown,"$89,105",17052
9,02467,Chestnut Hill,"$113,952",22796


In [33]:
wiki_mass_data['District'].replace('None', np.nan, inplace=True)
wiki_mass_data.dropna(subset=['District'], inplace=True)

wiki_mass_data.head()

,Zip Code,District,Median Household Income,Population
0,02110,Financial District,"$123,795",1486
1,02199,Prudential Center,"$107,159",1290
2,02210,Fort Point,"$111,061",1905
3,02109,North End,"$128,022",4277
4,02116,Back Bay/Bay Village,"$87,630",21318


In [34]:
wiki_mass_data.sort_values(by='Median Household Income', ascending=False)

,Zip Code,District,Median Household Income,Population
5,02108,Beacon Hill/Financial District,"$95,753",4155
8,02129,Charlestown,"$89,105",17052
4,02116,Back Bay/Bay Village,"$87,630",21318
11,02132,West Roxbury,"$82,421",27163
6,02114,Beacon Hill/West End,"$79,734",11933
13,02130,Jamaica Plain,"$74,198",36866
14,02127,South Boston,"$67,012",32547
10,02113,North End,"$64,413",7276
19,02131,Roslindale,"$61,099",30370
21,02136,Hyde Park,"$57,080",29219


In [35]:
wiki_mass_data.dtypes

Zip Code                   object
District                   object
Median Household Income    object
Population                  int64
dtype: object

In [36]:
df_geo = pd.read_csv('bostonzips2.csv')
df_geo.columns = ['Zip Code', 'Latitude', 'Longitude']
df_geo.head()

,Zip Code,Latitude,Longitude
0,2113,42.365028,-71.056360
1,2125,42.316852,-71.058110
2,2266,42.338947,-70.919635
3,2284,42.338947,-70.919635
4,2210,42.347974,-71.044630


In [37]:
wiki_mass_data.dtypes

Zip Code                   object
District                   object
Median Household Income    object
Population                  int64
dtype: object

In [38]:
wiki_mass_data['Zip Code'] = wiki_mass_data['Zip Code'].astype('int')
wiki_mass_data.dtypes

Zip Code                    int64
District                   object
Median Household Income    object
Population                  int64
dtype: object

In [39]:
df_pos = pd.merge(wiki_mass_data, df_geo, on=['Zip Code'], how='inner')
neighborhoods = df_pos[['Zip Code', 'District', 'Latitude', 'Longitude', 'Median Household Income']].copy()
                        
neighborhoods

,Zip Code,District,Latitude,Longitude,Median Household Income
0,2110,Financial District,42.356532,-71.05365,"$123,795"
1,2199,Prudential Center,42.347130,-71.08234,"$107,159"
2,2210,Fort Point,42.347974,-71.04463,"$111,061"
3,2109,North End,42.361477,-71.05417,"$128,022"
4,2116,Back Bay/Bay Village,42.349622,-71.07372,"$87,630"
5,2108,Beacon Hill/Financial District,42.357903,-71.06408,"$95,753"
6,2114,Beacon Hill/West End,42.361792,-71.06774,"$79,734"
7,2111,Chinatown/Financial District/Leather District,42.349838,-71.06101,"$44,758"
8,2113,North End,42.365028,-71.05636,"$64,413"
9,2118,South End,42.338724,-71.07276,"$50,000"


In [40]:
address = 'Boston, MA'

geolocator = Nominatim(user_agent='Bos_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The Geographical Coordinates of Boston are {}, {}.'.format(latitude, longitude))

The Geographical Coordinates of Boston are 42.3602534, -71.0582912.


In [41]:
map_boston = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, district, zcode in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['District'], neighborhoods['Zip Code']):
    label = '{}, {}'.format(district, zcode)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_boston)
    
map_boston

In [42]:
NorthEnd_data = neighborhoods[neighborhoods['District'] == 'North End'].reset_index(drop=True)
NorthEnd_data.head()

,Zip Code,District,Latitude,Longitude,Median Household Income


In [43]:
CLIENT_ID = 'PYHXTJFESZJO4BOA3DGGE1P0EJKYEBNZEHHGDXE50Q5MNYF4' # your Foursquare ID
CLIENT_SECRET = 'BC4VTK5IVYQZ5BDKLNYRCDUKHTN5LOSCT3CDVFNAHNTK1S4R' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: PYHXTJFESZJO4BOA3DGGE1P0EJKYEBNZEHHGDXE50Q5MNYF4
CLIENT_SECRET:BC4VTK5IVYQZ5BDKLNYRCDUKHTN5LOSCT3CDVFNAHNTK1S4R


In [44]:
neighborhoods.loc[3, 'District']

'North End '

In [45]:
neighborhood_latitude = neighborhoods.loc[3, 'Latitude']
neighborhood_longitude = neighborhoods.loc[3, 'Longitude']

neighborhood_name = neighborhoods.loc[3, 'District']

print('Latitude and Longitude values of {} are {}, {}'.format(neighborhood_name, neighborhood_latitude, neighborhood_longitude))

Latitude and Longitude values of North End  are 42.361477, -71.05417


In [46]:
LIMIT = 100

radius = 500

url= 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    neighborhood_latitude,
    neighborhood_longitude,
    radius,
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=PYHXTJFESZJO4BOA3DGGE1P0EJKYEBNZEHHGDXE50Q5MNYF4&client_secret=BC4VTK5IVYQZ5BDKLNYRCDUKHTN5LOSCT3CDVFNAHNTK1S4R&v=20180605&ll=42.361477,-71.05417&radius=500&limit=100'

In [47]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fcb132e67bf527854bf8bfe'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'North End',
  'headerFullLocation': 'North End, Boston',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 171,
  'suggestedBounds': {'ne': {'lat': 42.3659770045, 'lng': -71.04809130119621},
   'sw': {'lat': 42.3569769955, 'lng': -71.06024869880379}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4a1b3cd9f964a520db7a1fe3',
       'name': 'Quincy Market',
       'location': {'address': '1 Faneuil Hall Sq',
        'crossStreet': 'btwn Clinton & Chatham',
        'lat': 42.36010572657766,
        'lng': -71.054880997

In [48]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [49]:
venues = results['response']['groups'][0]['items']

nearby_venues = json_normalize(venues)

filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues = nearby_venues.loc[:, filtered_columns]

nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(20)

<ipython-input-49-916487b4f6e8>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues)


,name,categories,lat,lng
0,Quincy Market,Historic Site,42.360106,-71.054881
1,North End Park,Park,42.362488,-71.056477
2,Bricco Salumeria and Pasta Shop,Deli / Bodega,42.363071,-71.055041
3,The Rose Kennedy Greenway - Mothers Walk,Park,42.362640,-71.056407
4,Christopher Columbus Park,Park,42.361027,-71.051451
5,Thinking Cup,Coffee Shop,42.363221,-71.055124
6,Limoncello,Italian Restaurant,42.363353,-71.053827
7,Newbury Comics,Record Shop,42.360643,-71.054413
8,Improv Asylum Theatre,Comedy Club,42.362931,-71.055404
9,North End Yoga,Yoga Studio,42.363554,-71.054832


In [50]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [51]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([(
            name,
            lat,
            lng,
            v['venue']['name'],
            v['venue']['location']['lat'],
            v['venue']['location']['lng'],
            v['venue']['categories'][0]['name']) for v in results])
        
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood',
                            'Neighborhood Latitude',
                            'Neighborhood Longitude',
                            'Venue',
                            'Venue Latitude',
                            'Venue Longitude',
                            'Venue Category']
    
    return(nearby_venues)      

In [52]:
NorthEnd_venues = getNearbyVenues(names=neighborhoods['District'],
                                  latitudes=neighborhoods['Latitude'],
                                  longitudes=neighborhoods['Longitude']
                                 )

Financial District 
Prudential Center 
Fort Point 
North End 
Back Bay/Bay Village 
Beacon Hill/Financial District 
Beacon Hill/West End 
Chinatown/Financial District/Leather District 
North End 
South End 
South Boston 
East Boston 
Dorchester-Fields Corner 
Dorchester-Codman Square-Ashmont 
Dorchester-Uphams Corner-Savin Hill 
Allston-Harvard Business School 
Back Bay, Longwood, Museum of Fine Arts/Symphony Hall area 
Fenway-Kenmore 
Roxbury 
Dorchester-Mount Bowdoin 
Mission Hill 


In [53]:
NorthEnd_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Financial District,42.356532,-71.05365,Kane's Donuts,42.356209,-71.052895,Donut Shop
1,Financial District,42.356532,-71.05365,Trillium Garden On The Greenway,42.356540,-71.051100,Beer Garden
2,Financial District,42.356532,-71.05365,The Rose Kennedy Greenway,42.358227,-71.052055,Park
3,Financial District,42.356532,-71.05365,Casa Razdora,42.358231,-71.054741,Italian Restaurant
4,Financial District,42.356532,-71.05365,Equinox Franklin Street,42.356074,-71.054484,Gym


In [54]:
Boston_onehot = pd.get_dummies(NorthEnd_venues[['Venue Category']], prefix="", prefix_sep="")

Boston_onehot['Neighborhood'] = NorthEnd_venues['Neighborhood']

fixed_columns = [Boston_onehot.columns[-1]] + list(Boston_onehot.columns[:-1])
Boston_onehot = Boston_onehot[fixed_columns]

Boston_onehot.head()

,Yoga Studio,Accessories Store,African Restaurant,Airport,American Restaurant,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beer Bar,Beer Garden,Belgian Restaurant,Big Box Store,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Business Service,Café,Cambodian Restaurant,Caribbean Restaurant,Cemetery,Chinese Restaurant,Chocolate Shop,Church,Circus,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Cupcake Shop,Cycle Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food Court,Food Service,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gastropub,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Hockey Arena,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Insurance Office,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Marijuana Dispensary,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Venue,Neighborhood,New American Restaurant,Nightclub,Noodle House,Opera House,Other Repair Shop,Outdoor Sculpture,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Pool,Post Office,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Ski Chalet,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Tour Provider,Tourist Information Center,Track,Trail,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Financial District,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Financial District,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,Financial District,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [55]:
Boston_grouped = Boston_onehot.groupby('Neighborhood').mean().reset_index()
Boston_grouped

,Neighborhood,Yoga Studio,Accessories Store,African Restaurant,Airport,American Restaurant,Aquarium,Arepa Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auditorium,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Beer Bar,Beer Garden,Belgian Restaurant,Big Box Store,Bistro,Boat or Ferry,Bookstore,Boutique,Bowling Alley,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Burger Joint,Burrito Place,Bus Station,Business Service,Café,Cambodian Restaurant,Caribbean Restaurant,Cemetery,Chinese Restaurant,Chocolate Shop,Church,Circus,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Cupcake Shop,Cycle Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Electronics Store,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flower Shop,Food,Food Court,Food Service,Food Truck,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gastropub,Gift Shop,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Harbor / Marina,Health & Beauty Service,Historic Site,History Museum,Hockey Arena,Hostel,Hotel,Hotel Bar,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Insurance Office,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Lake,Latin American Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Mac & Cheese Joint,Malay Restaurant,Marijuana Dispensary,Market,Massage Studio,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Monument / Landmark,Movie Theater,Moving Target,Museum,Music Venue,New American Restaurant,Nightclub,Noodle House,Opera House,Other Repair Shop,Outdoor Sculpture,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Store,Pharmacy,Pilates Studio,Pizza Place,Playground,Plaza,Pool,Post Office,Pub,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shipping Store,Shoe Store,Shopping Mall,Skating Rink,Ski Chalet,Southern / Soul Food Restaurant,Souvenir Shop,Spa,Sporting Goods Shop,Sports Bar,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tea Room,Thai Restaurant,Theater,Thrift / Vintage Store,Tour Provider,Tourist Information Center,Track,Trail,Udon Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,Allston-Harvard Business School,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.029412,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.029412,0.000000,0.029412,0.00000,0.000000,0.000000,0.000000,0.00,0.000000,0.000,0.000000,0.029412,0.000000,0.000000,0.000000,0.029412,0.000,0.00,0.029412,0.00000,0.029412,0.000000,0.029412,0.000000,0.000000,0.000,0.000000,0.00,0.000,0.000000,0.029412,0.000000,0.029412,0.000000,0.00,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.029412,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.029412,0.000000,0.0000,0.000000,0.029412,0.000000,0.000000,0.000000,0.000000,0.029412,0.058824,0.029412,0.0000,0.000000,0.000000,0.000,0.00,0.000000,0.000000,0.000,0.000000,0.029412,0.029412,0.00,0.000000,0.000000,0.000000,0.058824,0.000000,0.000000,0.00,0.000,0.00000,0.000000,0.000000,0.000000,0.029412,0.000000,0.000000,0.000,0.00,0.000000,0.000000,0.00,0.000000,0.029412,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000,0.000000,0.029412,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.058824,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.0

In [56]:
num_top_venues = 5

for hood in Boston_grouped['Neighborhood']:
    print("----"+hood+'----')
    temp = Boston_grouped[Boston_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue', 'freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Allston-Harvard Business School ----
                  venue  freq
0           Pizza Place  0.06
1   Japanese Restaurant  0.06
2  Gym / Fitness Center  0.06
3                  Park  0.06
4        Ice Cream Shop  0.03


----Back Bay, Longwood, Museum of Fine Arts/Symphony Hall area ----
           venue  freq
0    Pizza Place  0.12
1     Art Museum  0.12
2         Garden  0.06
3  Grocery Store  0.06
4    Coffee Shop  0.06


----Back Bay/Bay Village ----
                  venue  freq
0                   Spa  0.07
1  Gym / Fitness Center  0.05
2    Mexican Restaurant  0.04
3    Seafood Restaurant  0.04
4                   Gym  0.04


----Beacon Hill/Financial District ----
                     venue  freq
0              Pizza Place  0.05
1              Coffee Shop  0.05
2  New American Restaurant  0.03
3            Historic Site  0.03
4               Steakhouse  0.03


----Beacon Hill/West End ----
                 venue  freq
0   Italian Restaurant  0.05
1  American Restaurant  0.05


In [57]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [58]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Boston_grouped['Neighborhood']

for ind in np.arange(Boston_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Boston_grouped.iloc[ind, :], num_top_venues)
    
neighborhoods_venues_sorted

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Allston-Harvard Business School,Park,Gym / Fitness Center,Pizza Place,Japanese Restaurant,Bookstore,Clothing Store,Bar,Fried Chicken Joint,Lingerie Store,Pool
1,"Back Bay, Longwood, Museum of Fine Arts/Sympho...",Pizza Place,Art Museum,Coffee Shop,Garden,Grocery Store,History Museum,Burger Joint,Baseball Field,Korean Restaurant,Lounge
2,Back Bay/Bay Village,Spa,Gym / Fitness Center,Seafood Restaurant,Gym,Hotel,Mexican Restaurant,Cosmetics Shop,Women's Store,Salon / Barbershop,Clothing Store
3,Beacon Hill/Financial District,Pizza Place,Coffee Shop,Historic Site,New American Restaurant,Italian Restaurant,Sandwich Place,Plaza,Steakhouse,Restaurant,American Restaurant
4,Beacon Hill/West End,History Museum,American Restaurant,Hotel Bar,Food Truck,Pizza Place,Italian Restaurant,Yoga Studio,Park,Deli / Bodega,Cycle Studio
5,Chinatown/Financial District/Leather District,Chinese Restaurant,Bakery,Asian Restaurant,Sushi Restaurant,Theater,Performing Arts Venue,Hotel Bar,Café,Bubble Tea Shop,Coffee Shop
6,Dorchester-Codman Square-Ashmont,Deli / Bodega,Park,Yoga Studio,Restaurant,Gym / Fitness Center,Grocery Store,Liquor Store,Food,Fast Food Restaurant,Diner
7,Dorchester-Fields Corner,Pizza Place,Pharmacy,Donut Shop,Chinese Restaurant,Arts & Crafts Store,Liquor Store,Bank,Sandwich Place,Cosmetics Shop,Pet Store
8,Dorchester-Mount Bowdoin,Pharmacy,Supermarket,Caribbean Restaurant,Nightclub,Donut Shop,Men's Store,Farmers Market,Fast Food Restaurant,Shopping Mall,Fish & Chips Shop
9,Dorchester-Uphams Corner-Savin Hill,Bar,Pizza Place,Pub,Park,Vietnamese Restaurant,Convenience Store,Health & Beauty Service,Pharmacy,Indian Restaurant,Fried Chicken Joint


In [59]:
neighborhoods_venues_sorted[15:16]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,North End,Italian Restaurant,Seafood Restaurant,Park,Pizza Place,Bakery,Coffee Shop,Café,Historic Site,Sandwich Place,Hotel


In [60]:
neighborhoods_venues_sorted[17:18]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Roxbury,American Restaurant,Convenience Store,Business Service,African Restaurant,Park,Fish Market,Fish & Chips Shop,Fast Food Restaurant,Farmers Market,Falafel Restaurant


In [61]:
kclusters = 5
Boston_grouped_clustering = Boston_grouped.drop('Neighborhood', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Boston_grouped_clustering)
kmeans.labels_[0:10]

array([2, 0, 2, 2, 2, 2, 2, 1, 4, 1], dtype=int32)

In [67]:

Boston_merged = neighborhoods

Boston_merged = Boston_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'))
Boston_merged

,Zip Code,District,Latitude,Longitude,Median Household Income,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,2110,Financial District,42.356532,-71.05365,"$123,795",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2199,Prudential Center,42.347130,-71.08234,"$107,159",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2210,Fort Point,42.347974,-71.04463,"$111,061",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2109,North End,42.361477,-71.05417,"$128,022",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2116,Back Bay/Bay Village,42.349622,-71.07372,"$87,630",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2108,Beacon Hill/Financial District,42.357903,-71.06408,"$95,753",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2114,Beacon Hill/West End,42.361792,-71.06774,"$79,734",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2111,Chinatown/Financial District/Leather District,42.349838,-71.06101,"$44,758",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2113,North End,42.365028,-71.05636,"$64,413",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2118,South End,42.338724,-71.07276,"$50,000",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
